<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [27]</a>'.</span>

# MLonMCU

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tum-ei-eda/mlonmcu/blob/main/ipynb/Demo.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tum-ei-eda/mlonmcu/blob/main/ipynb/Demo.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
</table>

## Prerequisites

Before running this notebook, make sure to run the following steps:

1. Install the required software for your distribution:

   See main `README.md` file!
   
2. Clone this repository

   ```
   git clone https://github.com/tum-ei-eda/mlonmcu.git
   cd mlonmcu  # enter cloned repository
   ```
   
3. Setup and enter a new virtualenv (Python 3.7+) for using `mlonmcu`:

   ```
   python -m venv .venv
   source .venv/bin/activate
   ```
   
   Hint: You can also use a `conda` environment for this.
   
4. Install mlonmcu package

   ```
   make install  # alternative: python setup.py install
   ```
   
5. Install the required python packages:

   ```
   # pip install -r requirements.txt  # Done automatically via setup.py?
   pip install -r ipynb/requirements.txt
   ```
   
6. Launch this notebook in jupyter:

   ```
   jupyter notebook ipynb/Demo.ipynb
   ```

*Hint for Developers:* If you do not want to install MLonMCU first, you can just replace `mlonmcu` with `python -m mlonmcu.cli.main` in every command to just use the latest development version.

## Demonstration (Command Line)

### Preface

This guide should give an idea on the required steps to setup and use MLonMCU using the command line interface. Usage examples are provided as well.

### Initialize a MLonMCU environment

As MLonMCU requires a directory where dependencies, configurations, intermediate artifacts and results are stored, an environment needs to be initialized first. This also allows to use different configuration (e.g. enabled backends or supported features) for multiple environments.

Create and enter an environment diretcory first to initialize it afterwards:

In [1]:
# WARNING: execute this only once! Alternatively remove the environment directory beforehand to start from scratch.
!test -d /tmp/mlonmcu_env && echo "Skipped initialization of environment" \
  || mlonmcu init /tmp/mlonmcu_env --non-interactive

Initializing ML on MCU environment
Selected target directory: /tmp/mlonmcu_env
The directory does not exist! - Created directory.
Creating environment.yml based on template 'default'.
Skipping creation of virtual environment. (already inside one)


Initializing directories in environment: deps plugins
Finished. Please add `export MLONMCU_HOME=/tmp/mlonmcu_env` to your shell configuration to use it anywhere
Recommended next step: `mlonmcu setup` inside your virtual environment.
Optional: `mlonmcu setup -g`


When not using a notebook, the option `--non-interactive` can be omitted which allows to customize the initialization process interactively.

If the environment directory is not passed to the `mlonmcu` script, it will try to setup a `default` environment in the users home directory (On Ubuntu: `~/.config/mlonmcu/environments/default/`) instead. By providing a `--name` of the virtual environment if will be automatically registered in `~/.config/mlonmcy/environments.ini` for the current user which helps to find and distinguish multiple installed environments on a machine.

In the following the actual environment which should be used by the `mlonmcu` program has to be defined by the user either by defining the environment variable `MLONMCU_HOME` or by using the `-H` (`--home` or `--hint`) command line option. If none of this was done, the following rules are following to look for a suitable environment:

1. Check if `$MLONMCU_HOME` points to a valid environment
2. Use environment provided via the `-H` flag (path or registered name) if available
3. If the local working directory is the root of an environment, use this directory
4. If there is a default environment for the current user, fall back to this one instead

In [2]:
# Use this for bash: `export MLONMCU_HOME=/tmp/mlonmcu_env`
%env MLONMCU_HOME=/tmp/mlonmcu_env

env: MLONMCU_HOME=/tmp/mlonmcu_env


Registered environment can be listed via the command `mlonmcu env`. In addition there are possibilities to manually update entires in the environments file.

In [3]:
!mlonmcu env

Looking for user environments config file: /home/runner/.config/mlonmcu/environments.ini
No mlonmcu environments were found for the current user. You can create a new one using `mlonmcu init`


### Customizing the created MLonMCU environment (optional)

The environment was initialized by a template which can be found in the `templates/` directory of the repository chosen via the `--template` flag. Lets inspect the generated configuration file first:

In [4]:
!cat $MLONMCU_HOME/environment.yml

---
# The MLONMCU_HOME is filled in automatically when creating the environment
home: "/tmp/mlonmcu_env"
logging:
  level: DEBUG
  to_file: false
  rotate: false
cleanup:
  auto: true
  keep: 50
# Default locations for certain directoriescan be changed here
# Non-absolute paths will always be threated relative to the MLONMCU_HOME
paths:
  # Where the dependencies are downloaded and installed
  deps: deps
  # If logging to file is used keep logs in this directory
  logs: logs
  # Location where reports and artifacts are written to
  results: results
  # Directory where custom extensions can be integrated
  plugins: plugins
  # Directory for intermediate build products, should be located on a large enough drive
  temp: temp
  # A collection of models which will be used to look for models
  # The paths will be checked in the order defined here stopping at the first match
  # Non-existant paths will be skipped without throwing an error
  models:
    - "/tmp/mlonmcu_env/models"
    - "/home

The environment should work out of the box, but feel free to update the contents of the `environment.yml` file to achieve one of the following:

- Change the default paths used for dependencies, results, models,...
- Use different forks/versions of a repository
- Disable certain frameworks/backends to reduce the installation time in the next step.
- Define which features are supported for the given repositories
- Set default backends and targets
- Define a set of default configuration variables

To learn more about this, check out this document: `ENVIRONMENTS.md`

### Setting up all MLonMCU dependencies

Depending on the enabled backends and features, a different set of dependencies has to be installed. To reduce the management effort for the user, all these versions are installed and managed automatically using MLonMCU. To trigger this process, run the following in the command line. (This will take a very long time with all frameworks/targets/features enabled when invoked for the first time)

In [5]:
!mlonmcu setup

INFO - No cache found in deps directory
INFO - Installing dependencies...
INFO - Processing task: clone_espidf


INFO - Processing task: clone_etiss


INFO - Processing task: clone_muriscvnn


INFO - Processing task: install_riscv_gcc{'pext': False, 'vext': False}


INFO - Processing task: clone_spike_pk


...

INFO - Processing task: install_etiss{'dbg': True}


INFO - Processing task: install_tvm


INFO - Processing task: build_cv32e40p


INFO - Processing task: clean_etiss


INFO - Finished installing dependencies


Some things need to be considered here:
- Repositories are only cloned once, so upstream changes or updated urls/branches in the `environments.yml` are not detected automatically
- The flag `--rebuild` can be passed to `mlonmcu setup` to ensure that the setup routine of every dependency is triggered again. However this does not guarentee that all artifacts are properly updated. Also there is a `--task` option which allows to invoke individual tasks manually.
- To guarantee that the latest versions of a dependency are installed you can remove the `deps` directory manually or alternative use the provided `mlonmcu cleanup` utility. (Execute `mlonmcu cleanup --help` to learn more)
- In addition to the installed dependencies, a file named `cache.ini` can be found in the environments `deps/` directory. Here all paths to relevant dependencies are stored with their different sets of flags.

### Adding models to MLonMCU

By default, the `mlonmcu-models` repository (https://github.com/tum-ei-eda/mlonmcu-models) is cloned to each environment as its contents are designed to perfectly integrate into the MLonMCU flow. In addition a location in the users home directory (`~/.config/mlonmcu/models/` for Ubuntu) is also added to the search path as defined in the `environments.yml`. Feel free to add further model manually or alternatively clone a model-zoo (e.g. https://github.com/ARM-software/ML-zoo) to a random location on the disc and updating the list of model-paths in the environments' config file.

To list all available models in a environment, the `mlonmcu models` subcommand can be used:

In [6]:
!mlonmcu models

INFO - Loading environment cache from file
INFO - Successfully initialized cache


Models Summary

Paths:
    /tmp/mlonmcu_env/models
    /home/runner/.config/mlonmcu/models

Models:
    MobileNetV2
    MobileNetV2/MobileNet_V2
    SMD_Ori_tflite
    aww
    bigsine_quant
    catdog
    catdog/CNN_2L_model_quantized
    catdog/CNN_full_quantized
    catdog/CNN_full_quantized_70_
    catdog/CNN_model
    catdog2
    cifar10
    cnn_s_quantized
    layers/aww_layer0
    layers/aww_layer1
    layers/aww_layer10
    layers/aww_layer11
    layers/aww_layer12
    layers/aww_layer2
    layers/aww_layer3
    layers/aww_layer4
    layers/aww_layer5
    layers/aww_layer6
    layers/aww_layer7
    layers/aww_layer8
    layers/aww_layer9
    layers/resnet_layer0
    layers/resnet_layer1
    layers/resnet_layer10
    layers/resnet_layer11
    layers/resnet_layer12
    layers/resnet_layer13
    layers/resnet_layer14
    layers/resnet_layer15
    layers/resnet_layer2
    layers/resnet_layer3
    layers/resnet_layer4
    layers/resnet_layer5
    layers/resnet_layer6
    layers/resne

The listed "Paths" should match the search paths defined in the `environments.yml` and are processed in the provided order. If there are multiple models of the same name, only the first one will be added to MLonMCU. By passing the `--detailed` flag to the command, additional information will be shown for each entry.

TODO: allow to search/filter for models by passing an extra arg

In addition to individual models, predefined model-groups are also available. I.e. the MLPerf Tiny benchmark defined here:

In [7]:
!cat $MLONMCU_HOME/models/modelgroups.yml

---
tinymlperf:
  - aww
  - vww
  - resnet
  - toycar

minimal:
  - sine_model

tinymlbook:
  - sine_model  # hello_world
  - micro_speech
  - magic_wand

embench:
  - edn
  - crc32
  - wikisort
  - slre
  - qrduino
  - minver
  - picojpeg
  - tarfind
  - st
  - nsichneu
  - statemate
  - primecount
  - md5sum
  - nettle-sha256
  - nettle-aes
  - ud
  - matmult-int
  - aha-mont64
  - huffbench
  - cubic
  - nbody
  - sglib-combined

taclebench:
  - app/powerwindow
  - app/lift
  - sequential/epic
  - sequential/huff_dec
  - sequential/fmref
  - sequential/h264_dec
  - sequential/dijkstra
  - sequential/adpcm_dec
  - sequential/adpcm_enc
  - sequential/gsm_dec
  - sequential/rijndael_dec
  - sequential/g723_enc
  - sequential/huff_enc
  - sequential/statemate
  - sequential/susan
  - sequential/gsm_enc
  - sequential/ndes
  - sequential/audiobeam
  - sequential/rijndael_enc
  - sequential/cjpeg_transupp
  - sequential/ammunition
  - sequential/mpeg2
  - sequential/anagram
  - sequential

Feel free to manualy define further model-groups for your needs.

### Running the MLonMCU flow

The MLonMCU flow is a major part of the tool and therefore explained by several examples, starting with minimal use cases before discussing more complex examples later.

**Invoke a single model on the default backend/target**

In [8]:
!mlonmcu flow run resnet

INFO - Loading environment cache from file
INFO - Successfully initialized cache


INFO -  Processing stage LOAD
INFO -  Processing stage BUILD


ERROR - Traceback (most recent call last):
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/__init__.py", line 79, in <module>
    from . import micro
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/__init__.py", line 24, in <module>
    from .model_library_format import (
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/model_library_format.py", line 35, in <module>
    from ..relay import param_dict
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/__init__.py", line 29, in <module>
    from . import prelude
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/prelude.py", line 21, in <module>
    from tvm.relay.transform import ToANormalFormExpr
  File "/tmp/ml

INFO - Done processing runs
INFO - Report:
   Session  Run   Model Frontend Framework Backend          Platform Target  Incomplete Features                                             Config Postprocesses Comment  Failing                  Reason
0        0    0  resnet   tflite       tvm  tvmaot  [microtvm, mlif]  etiss        True       []  {'resnet.output_types': {'Identity_int8': 'int...            []       -     True  AssertionError @ BUILD
ERROR - At least one error occured!


While this looks pretty clean, in the background, there is a lot of thing being processed. You can find out more using the `--verbose` flag:

In [9]:
!mlonmcu flow run resnet --verbose

DEBUG - Cleaning up old sessions automaticaly
DEBUG - Restored 1 recent sessions
DEBUG - Enter MlonMcuContext
DEBUG - Locking context
INFO - Loading environment cache from file
INFO - Successfully initialized cache
DEBUG - Creating a new session with idx 1
DEBUG - Found match. Ignoring other files
DEBUG - Creating a new run with id 0


DEBUG - Found match. Ignoring other files
INFO - [session-1]  Processing stage LOAD
DEBUG - [session-1] [run-0] Continuing run from stage LOAD until stage LOAD
DEBUG - [session-1] [run-0] Processing stage LOAD
DEBUG - [session-1] [run-0] Exporting run to disk
DEBUG - [session-1] [run-0] Writing run file
INFO - [session-1]  Processing stage BUILD
DEBUG - [session-1] [run-0] Continuing run from stage BUILD until stage BUILD
DEBUG - [session-1] [run-0] Processing stage BUILD
DEBUG - - Executing: ('/tmp/Demo-YH5O/venv/bin/python3', '-m', 'tvm.driver.tvmc', 'compile', PosixPath('/tmp/mlonmcu_env/temp/sessions/1/runs/0/resnet.tflite'), '--target', 'c', '--target-c-mcpu', 'generic-rv32', '--target-c-model', 'etiss-rv32gc', '--runtime', 'crt', '--executor', 'aot', '--pass-config', 'tir.disable_vectorize=True', '--pass-config', 'tir.usmp.enable=False', '--input-shapes', 'input_1_int8:[1,32,32,3]', '--opt-level', '3', '--output', '/tmp/tmpt6igr017/default.tar', '-f', 'mlf', '--model-format', 'tf

ERROR - Traceback (most recent call last):
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/__init__.py", line 79, in <module>
    from . import micro
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/__init__.py", line 24, in <module>
    from .model_library_format import (
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/model_library_format.py", line 35, in <module>
    from ..relay import param_dict
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/__init__.py", line 29, in <module>
    from . import prelude
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/prelude.py", line 21, in <module>
    from tvm.relay.transform import ToANormalFormExpr
  File "/tmp/ml

INFO - [session-1] Done processing runs
INFO - Report:
   Session  Run   Model Frontend Framework Backend          Platform Target  Incomplete Features                                             Config Postprocesses Comment  Failing                  Reason
0        1    0  resnet   tflite       tvm  tvmaot  [microtvm, mlif]  etiss        True       []  {'resnet.output_types': {'Identity_int8': 'int...            []       -     True  AssertionError @ BUILD
ERROR - At least one error occured!
DEBUG - Exit MlonMcuContext
DEBUG - Cleaning up active sessions
DEBUG - Releasing lock on context


The report of a run is typically the first thing to check out after invoking MLonMCU. It will be automatically written to the `results` directory declared in the `environments.yml`:

In [10]:
!ls $MLONMCU_HOME/results/*.csv

/tmp/mlonmcu_env/results/unnamed_20241002T083937.csv
/tmp/mlonmcu_env/results/unnamed_20241002T083939.csv


Let's define some helper functions to print these reports as a table. Of course you could also just open them i.e. with Excel! 

In [11]:
from helpers import tabularize_latest_report

In the following line the results or the latest run should be printed as a Pandas dataframe:

In [12]:
tabularize_latest_report()

,Session,Run,Model,Frontend,Framework,Backend,Platform,Target,Incomplete,Features,Config,Postprocesses,Comment,Failing,Reason
0,1,0,resnet,tflite,tvm,tvmaot,"['microtvm', 'mlif']",etiss,True,[],{'resnet.output_types': {'Identity_int8': 'int...,[],-,True,AssertionError @ BUILD


The artifacts for every invocation of the MLonMCU flow can be found in directory inside the environment. Your can find the latest files like this:

In [13]:
# Install the `tree` package before running this cell!
# !tree -L 3 $MLONMCU_HOME/temp/sessions/latest/

However there is a more comfortable way to do this: `mlonmcu export`

In [14]:
!mkdir -p /tmp/some_directory
!mlonmcu export /tmp/some_directory --run -f # By default this exports the latest run of the latest session

INFO - Loading environment cache from file
INFO - Successfully initialized cache
Creating directory: /tmp/some_directory
Done


Check out `mlonmcu export --help` to learn learn how to get the most out of this command. There is also a utility called `mlonmcu cleanup` which helps to get rid of old files after exporting the useful ones.

There exist multiple stages which can be processed. The subcommand passed to `mlonmcu flow` defines the stage where the flow should stop. If you are only interested in intermediate artifacts of the process, use one of the following:

```
mlonmcu flow load  # only process the model with the frontend
mlonmcu flow build  # invoke the chosen backend to generate code
mlonmcu flow compile  # (cross-)compile the code for the chosen target
```

Further commands are available to achive further behavior:

```
mlonmcu flow test  # use provided model-data to invoke the models comparing their outputs with a golden reference
```

**Explicitly specify the backend and target to use for the flow**

In [15]:
!mlonmcu flow run sine_model --backend tvmaot --target host_x86

INFO - Loading environment cache from file
INFO - Successfully initialized cache


INFO - [session-2]  Processing stage LOAD
INFO - [session-2]  Processing stage BUILD


ERROR - Traceback (most recent call last):
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/__init__.py", line 79, in <module>
    from . import micro
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/__init__.py", line 24, in <module>
    from .model_library_format import (
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/model_library_format.py", line 35, in <module>
    from ..relay import param_dict
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/__init__.py", line 29, in <module>
    from . import prelude
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/prelude.py", line 21, in <module>
    from tvm.relay.transform import ToANormalFormExpr
  File "/tmp/ml

INFO - [session-2]  Processing stage COMPILE
WARNING - Skiping stage 'COMPILE' for failed run
INFO - [session-2]  Processing stage RUN
WARNING - Skiping stage 'RUN' for failed run
ERROR - All runs have failed to complete!
INFO - Postprocessing session report
INFO - [session-2] Done processing runs
INFO - Report:
   Session  Run       Model Frontend Framework Backend Platform    Target  Incomplete Features                                             Config Postprocesses Comment  Failing                  Reason
0        2    0  sine_model   tflite       tvm  tvmaot     mlif  host_x86        True       []  {'sine_model.output_types': {'Identity': 'floa...            []       -     True  AssertionError @ BUILD
ERROR - At least one error occured!


**Enable certain features for all defined runs**

In [16]:
!mlonmcu flow run toycar --backend tvmaot --target etiss_pulpino --feature unpacked_api

INFO - Loading environment cache from file
INFO - Successfully initialized cache


INFO - [session-3]  Processing stage LOAD
INFO - [session-3]  Processing stage BUILD


ERROR - Traceback (most recent call last):
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/__init__.py", line 79, in <module>
    from . import micro
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/__init__.py", line 24, in <module>
    from .model_library_format import (
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/model_library_format.py", line 35, in <module>
    from ..relay import param_dict
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/__init__.py", line 29, in <module>
    from . import prelude
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/prelude.py", line 21, in <module>
    from tvm.relay.transform import ToANormalFormExpr
  File "/tmp/ml

INFO - Postprocessing session report
INFO - [session-3] Done processing runs
INFO - Report:
   Session  Run   Model Frontend Framework Backend Platform         Target  Incomplete        Features                                             Config Postprocesses Comment  Failing                  Reason
0        3    0  toycar   tflite       tvm  tvmaot     mlif  etiss_pulpino        True  [unpacked_api]  {'toycar.output_types': {'Identity': 'int8'}, ...            []       -     True  AssertionError @ BUILD
ERROR - At least one error occured!


**Add additional configuration options to customize internal components**

In [17]:
!mlonmcu flow run resnet --backend tvmaot --config tvmaot.target_device=arm_cpu

INFO - Loading environment cache from file
INFO - Successfully initialized cache


INFO - [session-4]  Processing stage LOAD
INFO - [session-4]  Processing stage BUILD


ERROR - Traceback (most recent call last):
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/__init__.py", line 79, in <module>
    from . import micro
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/__init__.py", line 24, in <module>
    from .model_library_format import (
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/model_library_format.py", line 35, in <module>
    from ..relay import param_dict
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/__init__.py", line 29, in <module>
    from . import prelude
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/prelude.py", line 21, in <module>
    from tvm.relay.transform import ToANormalFormExpr
  File "/tmp/ml

INFO - Report:
   Session  Run   Model Frontend Framework Backend          Platform Target  Incomplete Features                                             Config Postprocesses Comment  Failing                  Reason
0        4    0  resnet   tflite       tvm  tvmaot  [mlif, microtvm]  etiss        True       []  {'resnet.output_types': {'Identity_int8': 'int...            []       -     True  AssertionError @ BUILD
ERROR - At least one error occured!


Instead of defining certain key-value pass on each command line, feel free to use the `vars:` property defined in the `environment.yml` to achieve the same result.

**Invoke multiple models on multiple backends/targets**

In [18]:
!time mlonmcu flow run resnet toycar --backend tvmaot --backend tflmi  --target etiss_pulpino

INFO - Loading environment cache from file
INFO - Successfully initialized cache


INFO - [session-5]  Processing stage LOAD


INFO - [session-5]  Processing stage BUILD


ERROR - Traceback (most recent call last):
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/__init__.py", line 79, in <module>
    from . import micro
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/__init__.py", line 24, in <module>
    from .model_library_format import (
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/model_library_format.py", line 35, in <module>
    from ..relay import param_dict
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/__init__.py", line 29, in <module>
    from . import prelude
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/prelude.py", line 21, in <module>
    from tvm.relay.transform import ToANormalFormExpr
  File "/tmp/ml

ERROR - Traceback (most recent call last):
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/__init__.py", line 79, in <module>
    from . import micro
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/__init__.py", line 24, in <module>
    from .model_library_format import (
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/model_library_format.py", line 35, in <module>
    from ..relay import param_dict
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/__init__.py", line 29, in <module>
    from . import prelude
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/prelude.py", line 21, in <module>
    from tvm.relay.transform import ToANormalFormExpr
  File "/tmp/ml

INFO - [session-5]  Processing stage COMPILE
WARNING - Skiping stage 'COMPILE' for failed run
WARNING - Skiping stage 'COMPILE' for failed run


INFO - [session-5]  Processing stage RUN
WARNING - Skiping stage 'RUN' for failed run
WARNING - Skiping stage 'RUN' for failed run
ERROR - EtissPulpinoTarget.get_ini_bool_config() got an unexpected keyword argument 'override'
Traceback (most recent call last):
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/session/run.py", line 1090, in process
    func()
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/session/run.py", line 739, in run
    artifacts = self.target.generate_artifacts(elf_artifact.path)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/target/target.py", line 209, in generate_artifacts
    artifacts, metrics = self.generate(elf)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/target/target.py", line 189, in generate
    metrics_, out, artifacts_ = self.get_metrics(elf, temp_dir, *args)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/platform/mlif/mlif_target.py", line 226, in get_metrics
    metrics, out, artifacts = super().get_metrics(elf, directory, handle_exi

INFO - Postprocessing session report
INFO - [session-5] Done processing runs
INFO - Report:
   Session  Run   Model Frontend Framework Backend Platform         Target Incomplete  Total ROM  Total RAM  ROM read-only  ROM code  ROM misc  RAM data  RAM zero-init data Features                                             Config Postprocesses Comment  Failing                  Reason
0        5    0  resnet   tflite       tvm  tvmaot     mlif  etiss_pulpino       True        NaN        NaN            NaN       NaN       NaN       NaN                 NaN       []  {'resnet.output_types': {'Identity_int8': 'int...            []       -     True  AssertionError @ BUILD
1        5    1  resnet   tflite      tflm   tflmi     mlif  etiss_pulpino        NaN   197058.0    68556.0       101904.0   95010.0     144.0    1720.0             66836.0       []  {'resnet.output_types': {'Identity_int8': 'int...            []       -     True         TypeError @ RUN
2        5    2  toycar   tflite       tvm  

Command exited with non-zero status 1
682.44user 18.49system 3:10.34elapsed 368%CPU (0avgtext+0avgdata 445904maxresident)k
22384inputs+140192outputs (17major+1666283minor)pagefaults 0swaps


**Exploit parallelism to reduce the processing time**

In [19]:
!time mlonmcu flow run resnet toycar --backend tvmaot --backend tflmi --target etiss_pulpino --parallel --config runs_per_stage=0 --config mlif.num_threads=2

INFO - Loading environment cache from file
INFO - Successfully initialized cache


INFO - [session-6]  Processing stage LOAD
INFO - [session-6]  Processing stage BUILD


ERROR - Traceback (most recent call last):
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/__init__.py", line 79, in <module>
    from . import micro
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/__init__.py", line 24, in <module>
    from .model_library_format import (
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/model_library_format.py", line 35, in <module>
    from ..relay import param_dict
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/__init__.py", line 29, in <module>
    from . import prelude
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/prelude.py", line 21, in <module>
    from tvm.relay.transform import ToANormalFormExpr
  File "/tmp/ml

INFO - [session-6]  Processing stage RUN
WARNING - Skiping stage 'RUN' for failed run
WARNING - Skiping stage 'RUN' for failed run
ERROR - EtissPulpinoTarget.get_ini_bool_config() got an unexpected keyword argument 'override'
Traceback (most recent call last):
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/session/run.py", line 1090, in process
    func()
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/session/run.py", line 739, in run
    artifacts = self.target.generate_artifacts(elf_artifact.path)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/target/target.py", line 209, in generate_artifacts
    artifacts, metrics = self.generate(elf)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/target/target.py", line 189, in generate
    metrics_, out, artifacts_ = self.get_metrics(elf, temp_dir, *args)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/platform/mlif/mlif_target.py", line 226, in get_metrics
    metrics, out, artifacts = super().get_metrics(elf, directory, handle_exi

ERROR - All runs have failed to complete!
INFO - Postprocessing session report
INFO - [session-6] Done processing runs
INFO - Report:
   Session  Run   Model Frontend Framework Backend Platform         Target Incomplete  Total ROM  Total RAM  ROM read-only  ROM code  ROM misc  RAM data  RAM zero-init data Features                                             Config Postprocesses Comment  Failing                  Reason
0        6    0  resnet   tflite       tvm  tvmaot     mlif  etiss_pulpino       True        NaN        NaN            NaN       NaN       NaN       NaN                 NaN       []  {'resnet.output_types': {'Identity_int8': 'int...            []       -     True  AssertionError @ BUILD
1        6    1  resnet   tflite      tflm   tflmi     mlif  etiss_pulpino        NaN   197058.0    68556.0       101904.0   95010.0     144.0    1720.0             66836.0       []  {'resnet.output_types': {'Identity_int8': 'int...            []       -     True         TypeError @ RUN
2 

Command exited with non-zero status 1
680.69user 18.18system 3:00.34elapsed 387%CPU (0avgtext+0avgdata 445524maxresident)k
16inputs+140192outputs (8major+1668596minor)pagefaults 0swaps


**Display progress information for long-running benchmarks**

**WARNING:** The progress output looks bit broken inside the Jupyter Notebook but should look fine on the command-line.

In [20]:
!mlonmcu flow run resnet toycar micro_speech --progress --backend tvmaot --backend tflmi --target etiss_pulpino --target host_x86 --parallel --config mlif.num_threads=2

INFO - Loading environment cache from file
INFO - Successfully initialized cache


Processing stage LOAD:   0%|                                                         | 0/12 [00:00s]


                                                                                                    
Processing stage BUILD:   0%|                                                        | 0/12 [00:00s]


Processing stage BUILD:  17%|█████████▎                                              | 2/12 [00:00s]

ERROR - Traceback (most recent call last):
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py", line 110, in _get_module_details
    __import__(pkg_name)
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/__init__.py", line 79, in <module>
    from . import micro
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/__init__.py", line 24, in <module>
    from .model_library_format import (
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/micro/model_library_format.py", line 35, in <module>
    from ..relay import param_dict
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/__init__.py", line 29, in <module>
    from . import prelude
  File "/tmp/mlonmcu_env/deps/src/tvm/python/tvm/relay/prelude.py", line 21, in <module>
    from tvm.relay.transform import ToANormalFormExpr
  File "/tmp/ml

...


Processing stage COMPILE:  25%|█████████████▌                                        | 3/12 [06:20s]


Processing stage COMPILE:  42%|██████████████████████▌                               | 5/12 [09:08s]


Processing stage RUN:   0%|                                                          | 0/12 [00:00s]WARNING - Skiping stage 'RUN' for failed run
WARNING - Skiping stage 'RUN' for failed run
WARNING - Skiping stage 'RUN' for failed run
WARNING - Skiping stage 'RUN' for failed run
WARNING - Skiping stage 'RUN' for failed run
WARNING - Skiping stage 'RUN' for failed run
ERROR - EtissPulpinoTarget.get_ini_bool_config() got an unexpected keyword argument 'override'
Traceback (most recent call last):
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/session/run.py", line 1090, in process
    func()
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/session/run.py", line 739, in run
    artifacts = self.target.generate_artifacts(elf_artifact.path)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/target/target.py", line 209, in generate_artifacts
    artifacts, metrics = self.generate(elf)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/target/target.py", line 189, in generate
    metrics_,

ERROR - EtissPulpinoTarget.get_ini_bool_config() got an unexpected keyword argument 'override'
Traceback (most recent call last):
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/session/run.py", line 1090, in process
    func()
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/session/run.py", line 739, in run
    artifacts = self.target.generate_artifacts(elf_artifact.path)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/target/target.py", line 209, in generate_artifacts
    artifacts, metrics = self.generate(elf)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/target/target.py", line 189, in generate
    metrics_, out, artifacts_ = self.get_metrics(elf, temp_dir, *args)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/platform/mlif/mlif_target.py", line 226, in get_metrics
    metrics, out, artifacts = super().get_metrics(elf, directory, handle_exit=_handle_exit)
  File "/home/runner/work/mlonmcu/mlonmcu/mlonmcu/target/riscv/etiss.py", line 670, in get_metrics
    out_, artifa

INFO - Postprocessing session report
INFO - [session-7] Done processing runs
INFO - Report:
    Session  Run         Model Frontend Framework Backend Platform         Target Incomplete  Total ROM  Total RAM  ROM read-only  ROM code  ROM misc  RAM data  RAM zero-init data  End-to-End Runtime [s]  Total Cycles  Total Runtime [s] Validation Features                                             Config Postprocesses Comment Failing                  Reason
0         7    0        resnet   tflite       tvm  tvmaot     mlif  etiss_pulpino       True        NaN        NaN            NaN       NaN       NaN       NaN                 NaN                     NaN           NaN                NaN        NaN       []  {'resnet.output_types': {'Identity_int8': 'int...            []       -    True  AssertionError @ BUILD
1         7    1        resnet   tflite      tflm   tflmi     mlif  etiss_pulpino        NaN   197058.0    68556.0       101904.0   95010.0     144.0    1720.0             66836.0     

**Inspecting and exporting results and intermediate artifacts**

In [21]:
# !mlonmcu export --session 0 --run 0 run.zip  # Export run 0 from session 0 to a zip archive
# !mlonmcu export --session 1 session.zip  # Export every run of session 1 to a zip archive
# !mlonmcu export session/  # Export the latest session to a directory

**Cleanup an MLonMCU environment**

In [22]:
# !mlonmcu cleanup  # Interactively clean sessions
# !mlonmcu cleanup --force  # Clean all sessions
# !mlonmcu cleanup --keep 10 --force  # Keep only the latest 10 sessions
# !mlonmcu cleanup --dependencies  # Remove every installed dependency

**Internal command line interfaces**

In [23]:
!python -m mlonmcu.target.riscv.etiss_pulpino -h

/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py:126: RuntimeWarning: 'mlonmcu.target.riscv.etiss_pulpino' found in sys.modules after import of package 'mlonmcu.target.riscv', but prior to execution of 'mlonmcu.target.riscv.etiss_pulpino'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
usage: etiss_pulpino.py [-h] {exec,inspect} ...

ML on MCU Target

positional arguments:
  {exec,inspect}

options:
  -h, --help      show this help message and exit


In [24]:
!python -m mlonmcu.flow.tvm.backend.tvmaot -h

/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/runpy.py:126: RuntimeWarning: 'mlonmcu.flow.tvm.backend.tvmaot' found in sys.modules after import of package 'mlonmcu.flow.tvm.backend', but prior to execution of 'mlonmcu.flow.tvm.backend.tvmaot'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
usage: tvmaot.py [-h] [--output DIR] [--verbose] [--print] [-f FEATURE]
                 [-c KEY=VALUE [KEY=VALUE ...]]
                 MODEL

Run tvmaot backend

positional arguments:
  MODEL                 Model to process

options:
  -h, --help            show this help message and exit
  --output DIR, -o DIR  Output directory/file (default: /home/runner/work/mlonmcu/mlonmcu/ipynb/out)
  --verbose, -v         Print detailed messages for easier debugging (default: False)
  --print, -p           Print the generated code to the command line instead (default: False)
  -f FEATURE, --feature FEATURE
                        Enabled features for the backend (default: Non

## Demonstration (Python API)

### Preface

While it is also possible to initialize and setup an MLonMCU environment via Python, it is highly recommended to use the provided command line interfaces for these steps. The following examples demonstrate how to use the MLonMCU flow from within a Python program which enables more customizations of the internal features via scripting.

In [25]:
%env MLONMCU_HOME=/tmp/mlonmcu_env

env: MLONMCU_HOME=/tmp/mlonmcu_env


### Open a MLonMCU context to load the environment

In [26]:
import mlonmcu.context

with mlonmcu.context.MlonMcuContext() as context:
    pass

INFO - Loading environment cache from file


INFO - Successfully initialized cache


### Running the MLonMCU flow

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [27]:
import mlonmcu.context
from mlonmcu.session.run import RunStage

MODEL_NAMES = ["magic_wand", "micro_speech", "toycar"]

with mlonmcu.context.MlonMcuContext() as context:
    session = context.create_session()
    for model_name in MODEL_NAMES:
        run = session.create_run(features=[], config={})
        run.add_frontend_by_name("tflite", context=context)
        run.add_model_by_name(model_name, context=context)
        run.add_backend_by_name("tvmaot", context=context)
        run.add_platform_by_name("mlif", context=context)
        run.add_target_by_name("etiss_pulpino", context=context)
    session.process_runs(until=RunStage.RUN, context=context)
    report = session.get_reports()

INFO - Loading environment cache from file


INFO - Successfully initialized cache


AssertionError: Session needs to be opened first

Now we have the dataframe of the run we just defined and right in out python environment so we can have more fun with it.

In [ ]:
report.df

Let's add a quick visualization:

In [ ]:
import matplotlib.pyplot as plt

COLS = ["Cycles", "Total ROM", "Total RAM"]

fig, axes = plt.subplots(ncols=len(COLS))
plt.rcParams["figure.figsize"] = (15, 3)  # (w, h)


for i, col in enumerate(COLS):
    new_df = report.df[[col]].astype(float)
    new_df.index = MODEL_NAMES
    new_df.plot(kind="bar", ax=axes[i])